In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from time import time
import pandas as pd
from tqdm.notebook import tqdm
from itertools import permutations, combinations
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
main_dir = "/content/drive/My Drive/AaDS/"

# Code

In [ ]:
def load_dist_matrix(textfile, nodes_no):
  dists = np.loadtxt(textfile, skiprows=1)
  if nodes_no > dists.shape[0]:
    nodes_no = dists.shape[0]
  if nodes_no > 11:
    nodes_no = 11
  dists = dists[:nodes_no, :nodes_no]
  return dists

## Brute force

In [ ]:
def bf_tsp(infile, nodes_no):
  dists = load_dist_matrix(infile, nodes_no)
  t = time()
  nodes = np.arange(dists.shape[0])

  permut = list(permutations(nodes))
  min_dist = np.Inf
  shortest_path = tuple()

  for p in permut:
    d = 0
    p = p + (p[0],)
    for i in range(len(p)-1):
      d = d + dists[p[i], p[i+1]]
    if d < min_dist:
      min_dist = d
      shortest_path = p
  
  return shortest_path, min_dist, time() - t

## Dynamic programming

In [ ]:
def get_path(dists, path_table, cur_node, cur_dist, cur_set, end_point=0):
  path = (end_point,)
  while cur_node != end_point:
    path = (cur_node, ) + path
    cur_set = cur_set - 2**cur_node
    cur_node = np.where((path_table[:, cur_set] + dists[:, cur_node]) == cur_dist)[0][0]
    cur_dist = path_table[cur_node, cur_set]
  return (end_point,) +path 

def dp_tsp(infile, nodes_no):
  dists = load_dist_matrix(infile, nodes_no)
  t = time()
  nodes = np.arange(dists.shape[0])

  combinations_no = 2**nodes_no
  path_table = np.ones((nodes_no, combinations_no))* np.Inf
  # row - distance of row-th element from all combinations
  # column - distances of elements from particular combination
  # row indexes - by nodes number
  # column indexes - if element i is in the combination, then 
  # 2^i is added to the idx sum, i.e.:
  # idx 0 -> empty set
  # idx 1 -> {0}
  # idx 3 -> {0, 1}L
  # idx 2**nodes_no - 1 - set of all nodes
  path_table[0, 1] = 0 # distance of 0 from set {0}
  for set_size in range(1, nodes_no): # skipping empty set
    for S in combinations(range(1, nodes_no), set_size): 
      S = (0, ) + S # adding 0 as starting node
      set_index = np.sum(np.array([2])**np.array(S))
      for node in S:
        if node != 0: # because it is a starting point
          for prev_node in S: 
            # removing the current node from the set 
            # to get the index of set of predecessors
            connection_index = set_index - (2**node) # index of S - {node}
            # analysing all candidates for previous node
            # except the current node, cause no looping
            if node != prev_node:
              # set dist as minimum of current distance
              # and distance from prev_node summed with shortest path of S - {node}
              path_table[node][set_index] = min(path_table[node][set_index],
                                                path_table[prev_node][connection_index]+dists[prev_node, node])
  # setting up min_dist, reading shortest path
  end_point = 0
  last_set = combinations_no - 1

  min_dist = np.min(path_table[:, last_set]+dists[end_point, :]) 
  prev_point = np.argmin(path_table[:, last_set]+dists[end_point, :]) 
  prev_dist = path_table[prev_point, last_set]

  shortest_path = get_path(dists, path_table, prev_point, 
                           prev_dist, last_set, end_point)

  return shortest_path, min_dist, time() - t

## Google OR-Tools helper functions

In [ ]:
def create_data_model_tsp(textfile, nodes_no):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = load_dist_matrix(textfile, nodes_no).tolist()
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [ ]:
def get_routes_tsp(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an tuole."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  if any(isinstance(el, list) for el in routes):
    return tuple(routes[0])
  return tuple(routes)

In [ ]:
def get_distance_tsp(dists, path):
  min_dist = 0
  for i in range(1, len(path)):
    min_dist += dists[path[i-1]][path[i]]
  return min_dist

In [ ]:
def ortools_tsp(infile, nodes_no):
    # Instantiate the data problem.
    data = create_data_model_tsp(infile, nodes_no)
    t = time()
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
      # Get solution
      path = get_routes_tsp(solution, routing, manager)
      min_dist = get_distance_tsp(data['distance_matrix'], path)
      return path, min_dist, time() - t

## Google OR-tools: CNC drill

In [ ]:
def load_data_cnc(textfile, nodes_no):
  points = np.loadtxt(textfile, skiprows=1)
  if nodes_no > points.shape[0]:
    nodes_no = points.shape[0]
  points = points[:nodes_no, :]
  points = points.tolist()
  points = [tuple(coords) for coords in points]
  return points

In [ ]:
def create_data_model_cnc(textfile, nodes_no):
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    data['locations'] = load_data_cnc(textfile, nodes_no)
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [ ]:
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int(
                    math.hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))))
    return distances

In [ ]:
def print_solution_cnc(manager, routing, solution, outfile, nodes_no, points):
    """Prints solution on console."""
    index = routing.Start(0)
    with open(outfile, 'w') as f:
      f.write('{}\n'.format(nodes_no))
      while not routing.IsEnd(index):
        point = points[manager.IndexToNode(index)]
        f.write('{} {}\n'.format(int(point[0]), int(point[1])))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
      
      point = points[manager.IndexToNode(index)]
      f.write('{} {}\n'.format(int(point[0]),int(point[1])))

In [ ]:
def tsp_cnc(textfile, nodes_no, outfile, version=1):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model_cnc(textfile, nodes_no)
    t = time()
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    distance_matrix = compute_euclidean_distance_matrix(data['locations'])

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    if version==1:
      # Setting first solution heuristic.
      search_parameters = pywrapcp.DefaultRoutingSearchParameters()
      search_parameters.first_solution_strategy = (
          routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    else:
      # Setting second solution heuristic.
      search_parameters = pywrapcp.DefaultRoutingSearchParameters()
      search_parameters.local_search_metaheuristic = (
          routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
      search_parameters.time_limit.seconds = 30
      search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution_cnc(manager, routing, solution, outfile, nodes_no,
                           data['locations'])
    return solution.ObjectiveValue(), time() - t

# Testing

In [ ]:
infiles = ["g4.txt", "g13.txt", "g13.txt", "g13.txt", "g13.txt"]
nodes_no = [4, 4, 9, 10, 11]
measures = ["Input", "Points", 
            "Path BF", "Path DP", 
            "Length BF", "Length DP",
            "Times BF", "Times DP"]

ort_measures = ["Input", "Points", 
            "Path ORT", "Length ORT", "Times ORT"]

cnc_measures = ["Points", "Out ORv1", "Out ORv2", "Length ORv1", "Length ORv2", "Times ORv1", "Times ORv2"]

## Implementation TSP

In [ ]:
paths_bf = []
dists_bf = []
times_bf = []

paths_dp = []
dists_dp = []
times_dp = []

In [ ]:
for i in tqdm(range(len(infiles))):
  sp, min_dist, t = bf_tsp(main_dir+infiles[i], nodes_no[i])
  paths_bf.append(sp)
  dists_bf.append(min_dist)
  times_bf.append(t)

  sp, min_dist, t = dp_tsp(main_dir+infiles[i], nodes_no[i])
  paths_dp.append(sp)
  dists_dp.append(min_dist)
  times_dp.append(t)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(list(zip(infiles, nodes_no,
                           paths_bf, paths_dp,
                           dists_bf, dists_dp, 
                           times_bf, times_dp)),
               columns=measures)
df

,Input,Points,Path BF,Path DP,Length BF,Length DP,Times BF,Times DP
0,g4.txt,4,"(0, 1, 3, 2, 0)","(0, 2, 3, 1, 0)",80.0,80.0,0.006109,0.000751
1,g13.txt,4,"(0, 2, 1, 3, 0)","(0, 3, 1, 2, 0)",5000.0,5000.0,0.000146,0.000457
2,g13.txt,9,"(0, 5, 4, 1, 8, 6, 3, 2, 7, 0)","(0, 7, 2, 3, 6, 8, 1, 4, 5, 0)",6763.0,6763.0,2.173459,0.021804
3,g13.txt,10,"(0, 7, 2, 3, 6, 8, 1, 4, 5, 9, 0)","(0, 9, 5, 4, 1, 8, 6, 3, 2, 7, 0)",6811.0,6811.0,18.504475,0.043168
4,g13.txt,11,"(0, 7, 2, 3, 6, 8, 1, 4, 5, 10, 9, 0)","(0, 9, 10, 5, 4, 1, 8, 6, 3, 2, 7, 0)",7168.0,7168.0,169.129121,0.079088


## Google OR-Tools

In [ ]:
paths_ort = []
dists_ort = []
times_ort = []

In [ ]:
for i in tqdm(range(len(infiles))):
  sp, min_dist, t = ortools_tsp(main_dir+infiles[i], nodes_no[i])
  paths_ort.append(sp)
  dists_ort.append(min_dist)
  times_ort.append(t)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(list(zip(infiles, nodes_no,
                           paths_ort, dists_ort, 
                           times_ort)),
               columns=ort_measures)
df

,Input,Points,Path ORT,Length ORT,Times ORT
0,g4.txt,4,"(0, 1, 3, 2, 0)",80.0,0.002782
1,g13.txt,4,"(0, 2, 1, 3, 0)",5000.0,0.002422
2,g13.txt,9,"(0, 7, 2, 3, 6, 8, 1, 4, 5, 0)",6763.0,0.005698
3,g13.txt,10,"(0, 7, 2, 3, 6, 8, 1, 4, 5, 9, 0)",6811.0,0.005590
4,g13.txt,11,"(0, 7, 2, 3, 6, 8, 1, 4, 5, 10, 9, 0)",7168.0,0.005691


## CNC drill

In [ ]:
points_no = [35, 70, 140, 280]
outfile1 = ["tsp35v1.txt", "tsp70v1.txt", "tsp140v1.txt", "tsp280v1.txt"]
outfile2 = ["tsp35v2.txt", "tsp70v2.txt", "tsp140v2.txt", "tsp280v2.txt"]
infile_cnc = "c280.txt"

length_v1 = []
length_v2 = []
times_v1 = []
times_v2 = []

In [ ]:
for i in tqdm(range(len(points_no))):
  length, t = tsp_cnc(main_dir+infile_cnc, 
                      points_no[i], main_dir+outfile1[i],version=1)
  length_v1.append(length)
  times_v1.append(t)
  length, t = tsp_cnc(main_dir+infile_cnc, 
                      points_no[i], main_dir+outfile2[i],version=2)
  length_v2.append(length)
  times_v2.append(t)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(list(zip(points_no, outfile1, outfile2,
                           length_v1, length_v2, times_v1, times_v2)),
               columns=cnc_measures)
df

,Points,Out ORv1,Out ORv2,Length ORv1,Length ORv2,Times ORv1,Times ORv2
0,35,tsp35v1.txt,tsp35v2.txt,502,502,0.039076,30.007834
1,70,tsp70v1.txt,tsp70v2.txt,810,806,0.106810,30.013964
2,140,tsp140v1.txt,tsp140v2.txt,1457,1457,0.792734,30.018040
3,280,tsp280v1.txt,tsp280v2.txt,2790,2672,1.765590,30.051048
